In [1]:
import pandas as pd
import numpy as np

# Spotify Popularity Predictor (39%)

The goal of this challenge is to create a model that predicts the popularity of a song based on its features.

The dataset contains a list of tracks with the following characteristics:
- `acousticness`: whether the track is acoustic
- `danceability`: describes how suitable a track is for dancing
- `duration_ms`: duration of the track in milliseconds
- `energy`: represents a perceptual measure of intensity and activity
- `explicit`: whether the track has explicit lyrics
- `id`: id for the track
- `instrumentalness`: predicts whether a track contains no vocals
- `key`: the key the track is in
- `liveness`: detects the presence of an audience in the recording
- `loudness`: the overall loudness of a track in decibels
- `mode`: modality of a track
- `name`: name of the track
- `popularity`: popularity of the track
- `release_date`: release date
- `speechiness`: detects the presence of spoken words in a track
- `tempo`: overall estimated tempo of a track in beats per minute
- `valence`: describes the musical positiveness conveyed by a track
- `artist`: artist who performed the track

# Model

## Data collection

📝 **Load the `spotify_popularity_train.csv` dataset from the provided URL**
- Display the first few rows
- Perform the basic cleaning operations (remove redundant lines, as well as those with missing values)
- Store the result in a `DataFrame` named `data`

In [2]:
url = "https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/spotify_popularity_train.csv"

In [3]:
data = pd.read_csv(url)
data.head(1)

,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,artist
0,0.654,0.499,219827,0.19,0,0B6BeEUd6UwFlbsHMQKjob,0.00409,7,0.0898,-16.435,1,Back in the Goodle Days,40,1971,0.0454,149.46,0.43,John Hartford


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52317 entries, 0 to 52316
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      52317 non-null  float64
 1   danceability      52317 non-null  float64
 2   duration_ms       52317 non-null  int64  
 3   energy            52317 non-null  float64
 4   explicit          52317 non-null  int64  
 5   id                52317 non-null  object 
 6   instrumentalness  52317 non-null  float64
 7   key               52317 non-null  int64  
 8   liveness          52317 non-null  float64
 9   loudness          52317 non-null  float64
 10  mode              52317 non-null  int64  
 11  name              52317 non-null  object 
 12  popularity        52317 non-null  int64  
 13  release_date      52317 non-null  object 
 14  speechiness       52317 non-null  float64
 15  tempo             52317 non-null  float64
 16  valence           52317 non-null  float6

In [5]:
data.dropna(inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52313 entries, 0 to 52316
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      52313 non-null  float64
 1   danceability      52313 non-null  float64
 2   duration_ms       52313 non-null  int64  
 3   energy            52313 non-null  float64
 4   explicit          52313 non-null  int64  
 5   id                52313 non-null  object 
 6   instrumentalness  52313 non-null  float64
 7   key               52313 non-null  int64  
 8   liveness          52313 non-null  float64
 9   loudness          52313 non-null  float64
 10  mode              52313 non-null  int64  
 11  name              52313 non-null  object 
 12  popularity        52313 non-null  int64  
 13  release_date      52313 non-null  object 
 14  speechiness       52313 non-null  float64
 15  tempo             52313 non-null  float64
 16  valence           52313 non-null  float6

🧪 **Run the following cell to save your results**

In [7]:
from nbresult import ChallengeResult

ChallengeResult(
    "data_cleaning",
    shape=data.shape).write()

## Simple model

📝 **Which sklearn's scoring [metric](https://scikit-learn.org/stable/modules/model_evaluation.html) should we use if we want to:**
- **Strongly penalize** largest errors
- Measure errors **in the same unit** than `popularity` 
- Is better when greater (metric_good_model > metric_bad_model)

👉 Store its exact name as `string` in the variable `scoring` below

🚨 You must use this metric for the rest of the challenge

In [8]:
scoring = 'neg_root_mean_squared_error'

**📝 Let's build a first simple linear model using only the numerical features in our dataset to start with**
- Build `X_simple` keeping only numerical features
- Build `y` your target containing the `popularity`

In [9]:
X_simple = data[['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',\
                 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',\
                 'speechiness', 'tempo', 'valence',]]
y = data[['popularity']]

In [10]:
X_simple.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52313 entries, 0 to 52316
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      52313 non-null  float64
 1   danceability      52313 non-null  float64
 2   duration_ms       52313 non-null  int64  
 3   energy            52313 non-null  float64
 4   explicit          52313 non-null  int64  
 5   instrumentalness  52313 non-null  float64
 6   key               52313 non-null  int64  
 7   liveness          52313 non-null  float64
 8   loudness          52313 non-null  float64
 9   mode              52313 non-null  int64  
 10  speechiness       52313 non-null  float64
 11  tempo             52313 non-null  float64
 12  valence           52313 non-null  float64
dtypes: float64(9), int64(4)
memory usage: 5.6 MB


### Holdout evaluation

**📝 Create the 4 variables `X_train_simple` `y_train`, `X_test_simple`, `y_test` with a 50% split with random sampling**

In [11]:
from sklearn.model_selection import train_test_split

X_train_simple, X_test_simple, y_train, y_test = train_test_split(X_simple,y,test_size=0.5)

**📝 Fit and evaluate a basic linear model (do not fine tune it) with this holdout method**
- Store your model true performance in a float variable `score_simple_holdout`

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import math

preproc = StandardScaler()
lin_model = LinearRegression()

basic_model = Pipeline([('preproc',preproc),
                        ('model',lin_model)
                       ])

basic_model.fit(X_train_simple,y_train)

Pipeline(steps=[('preproc', StandardScaler()), ('model', LinearRegression())])

In [13]:
y_pred = basic_model.predict(X_test_simple)
score_simple_holdout = math.sqrt(mean_squared_error(y_test,y_pred))
score_simple_holdout

18.399993815466736

### Cross-validation evaluation

📝 **Let's be sure our score is representative**: 
- 5-times cross validate a basic linear model on the whole numeric dataset (`X_simple`, `y`)
- Do not fine tune your model
- Store your mean performance in a variable `score_simple_cv_mean` as a `float`
- Store the standard deviation of your performances in a float variable `score_simple_cv_std`

In [14]:
from sklearn.model_selection import cross_validate

cv = cross_validate(basic_model, X_simple, y, cv=5, scoring='neg_root_mean_squared_error')

In [15]:
cv['test_score']

array([-18.43649885, -18.46079317, -18.37189603, -18.47472207,
       -18.24571041])

In [16]:
score_simple_cv_mean = - cv['test_score'].mean()
score_simple_cv_std =  cv['test_score'].std()

In [17]:
score_simple_cv_mean

18.397924106389315

In [18]:
score_simple_cv_std

0.08388674341016651

🧪 **Run the following cell to save your results**

In [19]:
from nbresult import ChallengeResult

ChallengeResult(
    "simple_model",
    scoring=scoring,
    shape_train = X_train_simple.shape,
    score_simple_holdout=score_simple_holdout,
    score_simple_cv_mean=score_simple_cv_mean,
    score_simple_cv_std=score_simple_cv_std,
).write()

## Feature engineering

(From now on, we will stop using train/test split but cross-validation on the whole dataset instead)  

Let's try to improve performance using the feature `release_date`

**📝 Create `X_engineered` by adding a new column `year` to `X`, containing the release year of the track as `integer`**

In [20]:
import copy

In [21]:
release_year = pd.to_datetime(data['release_date']).dt.year
X_engineered = copy.copy(X_simple)
X_engineered['year']=release_year

In [22]:
X_engineered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52313 entries, 0 to 52316
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      52313 non-null  float64
 1   danceability      52313 non-null  float64
 2   duration_ms       52313 non-null  int64  
 3   energy            52313 non-null  float64
 4   explicit          52313 non-null  int64  
 5   instrumentalness  52313 non-null  float64
 6   key               52313 non-null  int64  
 7   liveness          52313 non-null  float64
 8   loudness          52313 non-null  float64
 9   mode              52313 non-null  int64  
 10  speechiness       52313 non-null  float64
 11  tempo             52313 non-null  float64
 12  valence           52313 non-null  float64
 13  year              52313 non-null  int64  
dtypes: float64(9), int64(5)
memory usage: 6.0 MB


📝 **Let's see how this impact the performance of our model.**
- Retrain the same simple linear model on numerical values only, adding the new feature `year`
- Save the mean cross-validated performance metric in a variable named `score_engineered` as a `float`
- Do not fine tune the model yet

In [23]:
cv = cross_validate(basic_model, X_engineered, y, cv=5, scoring='neg_root_mean_squared_error')

In [24]:
cv['test_score']

array([-17.48141143, -17.46493521, -17.38744409, -17.43882458,
       -17.21854704])

In [25]:
score_engineered = - cv['test_score'].mean()
score_engineered

17.398232470815284

🧪 **Run the following cell to save your results**

In [26]:
from nbresult import ChallengeResult

ChallengeResult("feature_engineering",
    cols = X_engineered.columns,
    years = X_engineered.get("year"),
    score_engineered=score_engineered
).write()

## Pipelining

Let's now look for maximum performance by creating a solid preprocessing pipeline.

**📝 Create a sklearn preprocessing [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and store it as `preproc`**

- Feel free to add any preprocessing steps you think of
- You may want to integrate your feature engineering for `year`
- You may also further improve it using the `ArtistPopularityTransformer` class given to you below
- Don't add any model to it yet

🚨 Advice: It is better for you to have a working pipeline (even simple one) rather than NO pipeline at all

In [27]:
# 👉 Do not hesitate to reload a clean new dataset if you need a fresh start.
# X = 
# y = 

X = copy.copy(data)[['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',\
                 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',\
                 'speechiness', 'tempo', 'valence','artist']]

release_year = pd.to_datetime(data['release_date']).dt.year
X['year']=release_year

y = data['popularity']

In [28]:
# We are giving you below a custom transformer that you may want to use in your pipeline (make sure you understanding it)

from sklearn.base import BaseEstimator, TransformerMixin

class ArtistPopularityTransformer(BaseEstimator, TransformerMixin):
    """
    Compute, as a new feature of the test set, the mean popularity of 
    all songs made by the artist on the train set.
    """

    def __init__(self):
        pass

    def fit(self, X, y=None):
        """
        process artist mean popularity from artists songs popularity
        process song global mean popularity
        """

        # process artist popularity
        self.artist_popularity = y.groupby(X.artist).agg("mean")
        self.artist_popularity.name = "artist_popularity"

        # process mean popularity
        self.mean_popularity = y.mean()

        return self

    def transform(self, X, y=None):
        """
        apply artist mean popularity vs song global mean popularity to songs
        """

        # inject artist popularity
        X_copy = X.merge(self.artist_popularity, how="left", left_on="artist", right_index=True)

        # fills popularity of unknown artists with song global mean popularity
        X_copy.replace(np.nan, self.mean_popularity, inplace=True)

        return X_copy[["artist_popularity"]]

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion


union = FeatureUnion([
                      ('artist_popularity', ArtistPopularityTransformer())
                     ])

pop_pipe = Pipeline([('pop', union),
                     ('scaler', StandardScaler())
                   ])

num_pipe = Pipeline([('scaler', StandardScaler())
                   ])

num_features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',\
                 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',\
                 'speechiness', 'tempo', 'valence', 'year']

preproc = ColumnTransformer([
                             ('pop_preproc', pop_pipe, ['artist']),
                             ('num_preproc', num_pipe, num_features),
                            ])

In [30]:
len(X.keys())

15

In [31]:
first_observation = preproc.fit_transform(X, y)[0]
len(first_observation)

15

**📝 Store the number of columns/feature after preprocessing your inputs in a variable `col_number`**

In [32]:
col_number = 15

🧪 **Run the following cells to save your results**

In [33]:
# Visually print your preproc
from sklearn import set_config; set_config(display='diagram')
preproc

ColumnTransformer(transformers=[('pop_preproc',
                                 Pipeline(steps=[('pop',
                                                  FeatureUnion(transformer_list=[('artist_popularity',
                                                                                  ArtistPopularityTransformer())])),
                                                 ('scaler', StandardScaler())]),
                                 ['artist']),
                                ('num_preproc',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['acousticness', 'danceability', 'duration_ms',
                                  'energy', 'explicit', 'instrumentalness',
                                  'key', 'liveness', 'loudness', 'mode',
                                  'speechiness', 'tempo', 'valence', 'year'])])

In [34]:
# Save your preproc
from nbresult import ChallengeResult

ChallengeResult(
    "preprocessing",
    col_number=col_number,
    first_observation = preproc.fit_transform(X, y)[0]
).write()

## Training

📝 **Time to fine tune your models**

- Add an **estimator** to your pipeline (only from Scikit-learn) 

- Train your pipeline and **fine-tune** (optimize) your estimator to maximize prediction score

- You must try to fine tune at least 2 different models: 
    - create one pipeline with a **linear model** of your choice
    - create one pipeline with an **ensemble model** of your choice

Then, 

- Save your two best 5-time cross-validated scores as _float_: `score_linear` and `score_ensemble`

- Save your two best trained pipelines as _Pipeline_ objects: `pipe_linear` and `pipe_ensemble`

### Linear

In [35]:
pipe_linear = Pipeline([ ('preproc', preproc),
                         ('linear_model', LinearRegression())
                       ])

cv_linear = cross_validate(pipe_linear, X, y, cv=5, scoring='neg_root_mean_squared_error')
cv_linear['test_score']

array([-13.71572367, -13.37446416, -13.45908332, -13.35645739,
       -13.38849818])

In [36]:
score_linear = -cv_linear['test_score'].mean()
score_linear

13.458845342613463

### Ensemble

In [37]:
from sklearn.ensemble import GradientBoostingRegressor

pipe_ensemble = Pipeline([ ('preproc', preproc),
                         ('linear_model', GradientBoostingRegressor())
                       ])

cv_ensemble = cross_validate(pipe_ensemble, X, y, cv=5, scoring='neg_root_mean_squared_error')
cv_ensemble['test_score']

array([-12.75116247, -12.27878333, -12.46161193, -12.31914165,
       -12.25134991])

In [38]:
score_ensemble = - cv_ensemble['test_score'].mean()
score_ensemble

12.41240985818427

🧪 **Run the following cells to save your results**

In [39]:
# Print below your best pipe for correction purpose
from sklearn import set_config; set_config(display='diagram')
pipe_linear

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pop_preproc',
                                                  Pipeline(steps=[('pop',
                                                                   FeatureUnion(transformer_list=[('artist_popularity',
                                                                                                   ArtistPopularityTransformer())])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['artist']),
                                                 ('num_preproc',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['acousticness',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'explicit',
                                                   'instrumentalness', 'key',
                                                   'liveness', 'loudness',
                                                   'mode', 'speechiness',
                                                   'tempo', 'valence',
                                                   'year'])])),
                ('linear_model', LinearRegression())])

In [40]:
# Print below your best pipe for correction purpose
pipe_ensemble

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pop_preproc',
                                                  Pipeline(steps=[('pop',
                                                                   FeatureUnion(transformer_list=[('artist_popularity',
                                                                                                   ArtistPopularityTransformer())])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['artist']),
                                                 ('num_preproc',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['acousticness',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'explicit',
                                                   'instrumentalness', 'key',
                                                   'liveness', 'loudness',
                                                   'mode', 'speechiness',
                                                   'tempo', 'valence',
                                                   'year'])])),
                ('linear_model', GradientBoostingRegressor())])

In [41]:
from nbresult import ChallengeResult

ChallengeResult("model_tuning",
    scoring = scoring,
    score_linear=score_linear,
    score_ensemble=score_ensemble).write()

## API 

Time to put a pipeline in production!

👉 Go to https://github.com/lewagon/data-certification-api and follow instructions

**This final part is independent from the above notebook**